In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import rerun as rr
import genjax
import os
import numpy as np
import jax.numpy as jnp
import jax
from b3d import Pose
import b3d
from tqdm   import tqdm

PORT = 8812
rr.init("object_prior_test")
rr.connect(addr=f'127.0.0.1:{PORT}')

In [4]:
path = os.path.join(b3d.get_assets_path(),
#  "shared_data_bucket/input_data/orange_mug_pan_around_and_pickup.r3d.video_input.npz")
# "shared_data_bucket/input_data/shout_on_desk.r3d.video_input.npz")
"shared_data_bucket/input_data/desk_ramen2_spray1.r3d.video_input.npz")
video_input = b3d.VideoInput.load(path)

In [5]:
image_width, image_height, fx,fy, cx,cy,near,far = np.array(video_input.camera_intrinsics_depth)
image_width, image_height = int(image_width), int(image_height)
fx,fy, cx,cy,near,far = float(fx),float(fy), float(cx),float(cy),float(near),float(far)

rgbs = video_input.rgb[::3] / 255.0
xyzs = video_input.xyz[::3]
# Resize rgbs to be same size as depth.
rgbs_resized = jnp.clip(jax.vmap(jax.image.resize, in_axes=(0, None, None))(
    rgbs, (video_input.xyz.shape[1], video_input.xyz.shape[2], 3), "linear"
), 0.0, 1.0)

point_cloud_og = xyzs[0].reshape(-1,3)
colors_og = rgbs_resized[0].reshape(-1,3)

In [6]:
import b3d.model2 as model2

In [ ]:
genjax.const

In [125]:
key = jax.random.PRNGKey(0)
intrinsics = (fx,fy, cx,cy,near,far)
subx, suby = 8, 8
p_occupancy = 0.4
C = genjax.Pytree.const
args = (
    C(image_width), C(image_height),
    C(subx), C(suby),
    C(intrinsics), p_occupancy
)
trace = model2.generate_object_voxel_mesh.simulate(key, args)

In [126]:
vertices, faces, vertex_colors, face_colors, point_centers, depth = trace.get_retval()

In [127]:
rr.log(
    "/3d/mesh",
    rr.Mesh3D(
        vertex_positions=vertices,
        indices=faces,
        vertex_colors=vertex_colors
    ),
    timeless=True
)

In [97]:
vertices[-1]

Array([-36.591602, -59.232773, 100.00007 ], dtype=float32)

In [91]:
vertices[100:120,]

Array([[-47.20476 , -72.08062 ,  99.99984 ],
       [-47.20476 , -72.08062 , 100.00016 ],
       [-47.20476 , -72.0803  ,  99.99984 ],
       [-47.20476 , -72.0803  , 100.00016 ],
       [-46.646496, -72.08063 ,  99.99983 ],
       [-46.646496, -72.08063 , 100.00017 ],
       [-46.646496, -72.08029 ,  99.99983 ],
       [-46.646496, -72.08029 , 100.00017 ],
       [-46.646168, -72.08063 ,  99.99983 ],
       [-46.646168, -72.08063 , 100.00017 ],
       [-46.646168, -72.08029 ,  99.99983 ],
       [-46.646168, -72.08029 , 100.00017 ],
       [-46.087868, -72.08059 ,  99.99987 ],
       [-46.087868, -72.08059 , 100.00013 ],
       [-46.087868, -72.08033 ,  99.99987 ],
       [-46.087868, -72.08033 , 100.00013 ],
       [-46.08761 , -72.08059 ,  99.99987 ],
       [-46.08761 , -72.08059 , 100.00013 ],
       [-46.08761 , -72.08033 ,  99.99987 ],
       [-46.08761 , -72.08033 , 100.00013 ]], dtype=float32)

In [83]:
far

100.0

In [56]:
(voxel_present, depth, colors, resolutions, point_centers) = trace.get_retval()
depth_flat = depth.reshape(-1)
colors_flat = colors.reshape(-1, 3)

In [59]:
resolutions.shape

(3072,)

In [61]:
voxel_present.shape

(48, 64)

In [62]:
vertices, faces, vertex_colors, face_colors = b3d.utils.make_mesh_from_point_cloud_and_resolution_2(
    point_centers,
    colors_flat,
    resolutions,
    voxel_present.reshape(-1)
)

In [18]:
voxel_present, depth, colors = trace.get_retval()

In [28]:
f = lambda m : m.match(lambda: -1.0, lambda x : x)
jax.vmap(jax.vmap(f, in_axes=(0,)), in_axes=(0,))(depth)

Array([[ 8.2909977e-03,  7.7206809e-03,  5.9520635e-03, ...,
         4.7618030e-03,  9.8284325e-03, -1.0000000e+00],
       [ 1.9602785e-03,  1.8960666e-03, -1.0000000e+00, ...,
         3.5102842e-03,  1.7511351e-03,  6.8051419e-03],
       [ 3.4411047e-03, -1.0000000e+00,  8.8439239e-03, ...,
         4.7888327e-03,  2.6385877e-03, -1.0000000e+00],
       ...,
       [-1.0000000e+00,  4.2594587e-03,  3.0224035e-03, ...,
         2.0125713e-03,  7.3168082e-03,  7.4895606e-03],
       [ 8.3255786e-03,  7.2128675e-03,  2.3591798e-04, ...,
         6.9076382e-04,  4.4467375e-03,  8.7554250e-03],
       [ 2.0493641e-03, -1.0000000e+00,  3.4507955e-03, ...,
        -1.0000000e+00,  2.1509863e-03,  5.5744122e-03]], dtype=float32)

In [24]:
genjax.Mask.match

BoundMethod(
  __func__=<wrapped function match>,
  __self__=Mask(flag=i32[48,64], value=f32[48,64])
)

In [ ]:
vertices, faces, vertex_colors, face_colors = trace.get_retval()